In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [111]:
df = pd.read_csv('train.csv')

In [112]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [113]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [114]:
df.Age = df.Age.fillna(df.Age.mean())

In [115]:
df.Embarked = df.Embarked.fillna(df.Embarked.mode())

In [116]:
df = df.drop(columns = ['Name'])
df = df.drop(columns = ['Ticket'])
df = df.drop(columns = ['Cabin'])

In [117]:
dummies = pd.get_dummies(df[['Sex', 'Embarked']], drop_first = True)

In [118]:
df = pd.concat([df, dummies], axis=1)

In [119]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,0,0,1
4,5,0,3,male,35.0,0,0,8.0500,S,1,0,1


In [120]:
X = df.drop(columns = ['Fare','Embarked','Survived','Sex'])
y = df.Survived

In [121]:
from sklearn.model_selection import train_test_split
X_egitim, X_test, y_egitim, y_test = train_test_split(X, y, test_size=0.2)
print("Eğitim veri kümesindeki satır sayısı :  {} ".format(len(X_egitim)))
print("Eğitim veri kümesindeki hedef sayısı :  {} ".format(len(y_egitim)))
print("Test veri kümesindeki satır sayısı :  {} ".format(len(X_test)))
print("Test veri kümesindeki hedef sayısı :  {} ".format(len(y_test)))

Eğitim veri kümesindeki satır sayısı :  712 
Eğitim veri kümesindeki hedef sayısı :  712 
Test veri kümesindeki satır sayısı :  179 
Test veri kümesindeki hedef sayısı :  179 


In [122]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_egitim, y_egitim)
tahmin_eğitim = model.predict(X_egitim)
tahmin_test = model.predict(X_test)
model.score(X_test, y_test)

0.7374301675977654

In [123]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=111)

In [124]:
parcalar = kf.split(X)
for num, (eğitim_index, test_index) in enumerate(parcalar):
    print("{}.Eğitim Kümesi Boyutu : {}".format(num+1,len(eğitim_index)))
    print("{}.Test Kümesi Boyutu   : {}".format(num+1,len(test_index)))
    print('-'*27)

1.Eğitim Kümesi Boyutu : 712
1.Test Kümesi Boyutu   : 179
---------------------------
2.Eğitim Kümesi Boyutu : 713
2.Test Kümesi Boyutu   : 178
---------------------------
3.Eğitim Kümesi Boyutu : 713
3.Test Kümesi Boyutu   : 178
---------------------------
4.Eğitim Kümesi Boyutu : 713
4.Test Kümesi Boyutu   : 178
---------------------------
5.Eğitim Kümesi Boyutu : 713
5.Test Kümesi Boyutu   : 178
---------------------------


In [125]:
from sklearn.model_selection import cross_validate, cross_val_score

In [126]:
cv = cross_val_score(estimator=lrm,
                     X=X,
                     y=y,
                     cv=10                    
                    )
print('Model Skorları            : ', cv, sep = '\n')

Model Skorları            : 
[0.81111111 0.8        0.7752809  0.83146067 0.7752809  0.7752809
 0.78651685 0.75280899 0.82022472 0.80681818]


In [127]:
from sklearn.model_selection import cross_val_predict
y_tahmin = cross_val_predict(estimator=lrm, X=X, y=y, cv=10)
print(y_tahmin[0:10])

[0 1 1 1 0 0 0 0 0 1]


In [128]:
logreg = LogisticRegression()
print(logreg.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'warn', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


### Grid Search

In [129]:
parametreler = {"C": [10 ** x for x in range (-5, 5, 1)],
                "penalty": ['l1', 'l2']
                }

In [130]:
from sklearn.model_selection import GridSearchCV
grid_cv = GridSearchCV(estimator=logreg,
                       param_grid = parametreler,
                       cv = 10
                      )
grid_cv.fit(X, y)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000, 10000],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [131]:
print("En iyi parametreler : ", grid_cv.best_params_)
print("En iyi skor         : ", grid_cv.best_score_)

En iyi parametreler :  {'C': 100, 'penalty': 'l1'}
En iyi skor         :  0.7968574635241302


In [132]:


sonuçlar = grid_cv.cv_results_
df = pd.DataFrame(sonuçlar)
df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004797,0.000979,0.001699,0.000640,1e-05,l1,"{'C': 1e-05, 'penalty': 'l1'}",0.611111,0.611111,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.613636,0.616162,0.002844,14
1,0.004896,0.001135,0.001600,0.000664,1e-05,l2,"{'C': 1e-05, 'penalty': 'l2'}",0.611111,0.611111,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.613636,0.616162,0.002844,14
2,0.005796,0.004932,0.001799,0.001248,0.0001,l1,"{'C': 0.0001, 'penalty': 'l1'}",0.611111,0.611111,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.613636,0.616162,0.002844,14
3,0.003199,0.000600,0.001297,0.000458,0.0001,l2,"{'C': 0.0001, 'penalty': 'l2'}",0.611111,0.611111,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.613636,0.616162,0.002844,14
4,0.004100,0.000832,0.001499,0.000496,0.001,l1,"{'C': 0.001, 'penalty': 'l1'}",0.611111,0.611111,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.617978,0.613636,0.616162,0.002844,14


In [133]:
df = df[['param_penalty','param_C', 'mean_test_score']]
df = df.sort_values(by='mean_test_score', ascending = False)
df

,param_penalty,param_C,mean_test_score
19,l2,10000,0.796857
14,l1,100,0.796857
17,l2,1000,0.795735
16,l1,1000,0.795735
15,l2,100,0.795735
13,l2,10,0.795735
12,l1,10,0.794613
10,l1,1,0.794613
11,l2,1,0.793490
18,l1,10000,0.793490
